In [12]:
import ee
import pandas as pd
import os
import xarray as xr
from datetime import datetime

In [2]:
era5land = pd.read_csv("ERA5LandInitialcondition.csv", index_col=[0])
era5land

,skin_temperature,soil_temperature_level_1,soil_temperature_level_2,soil_temperature_level_3,soil_temperature_level_4,volumetric_soil_water_layer_1,volumetric_soil_water_layer_2,volumetric_soil_water_layer_3,volumetric_soil_water_layer_4,latitude,longitude,image_date
BE-Lon,283.001587,281.349380,280.044296,280.035721,283.202637,0.410507,0.413544,0.396851,0.343674,50.551620,4.746234,2012-01-01
CZ-KrP,274.378357,273.626114,274.277328,277.239838,280.646927,0.345505,0.351242,0.364731,0.365387,49.573257,15.078773,2014-01-01
FI-Qvd,273.757843,274.273071,274.815628,276.230698,278.926910,0.308960,0.320221,0.326996,0.296204,60.295242,22.391607,2018-01-01
FR-Aur,283.609009,282.904068,282.298203,282.639236,288.144043,0.353378,0.336655,0.260178,0.274643,43.549650,1.106103,2012-01-01
FR-Gri,284.257446,283.060318,282.048203,281.248611,284.985840,0.395081,0.393890,0.352097,0.302551,48.844220,1.951910,2012-01-01
DE-RuS,282.357056,280.620865,279.247421,279.744705,283.062012,0.404297,0.401428,0.359100,0.328949,50.865906,6.447145,2012-01-01
IT-BCi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.523750,14.957444,2012-01-01
DK-Sor,275.067993,275.363052,275.825546,278.856033,281.813965,0.371368,0.376480,0.379166,0.360229,55.485870,11.644645,2012-01-01
FR-Fon,284.327759,283.073990,282.007187,281.109940,285.231934,0.384308,0.379318,0.313004,0.285812,48.476357,2.780096,2012-01-01
FR-Hes,276.278748,276.428848,276.798813,279.274994,281.965286,0.391190,0.398315,0.405212,0.395447,48.674100,7.064650,2014-01-01


In [3]:
output_path = "/home/khanalp/task1/data/ICOS/Initial_condition/"

In [4]:
# Convert the image_date column to datetime
era5land['image_date'] = pd.to_datetime(era5land['image_date'])

In [15]:
#Renaming them 
rename = {'skin_temperature':'skt',
          'soil_temperature_level_1':'stl1',
          'soil_temperature_level_2':'stl2',
          'soil_temperature_level_3':'stl3',
          'soil_temperature_level_4':'stl4',
          'volumetric_soil_water_layer_1':'swvl1',
          'volumetric_soil_water_layer_2':'swvl2',
          'volumetric_soil_water_layer_3':'swvl3',
          'volumetric_soil_water_layer_4':'swvl4'
         }

In [18]:
# Iterate over the rows and create xarray Datasets
for index, row in era5land.iterrows():
    # Convert the row to a dictionary
    row_dict = row.to_dict()

    station_name = index
    if station_name not in ["IT-BCi"]:
        print(f"Processing {station_name}")
        # Extract latitude, longitude, and image_date
        lat = row_dict.pop('latitude')
        lon = row_dict.pop('longitude')
        date = row_dict.pop('image_date')
        print(date)
        # Create an xarray Dataset from the dictionary
        xds = xr.Dataset(row_dict)
        
        # Add latitude, longitude, and image_date as coordinates
        xds.coords['latitude'] = lat
        xds.coords['longitude'] = lon
        xds.coords['time'] = date
        # Select variables and rename them.
        xds_renamed = xds.rename(rename)
        # Iterate over each data variable in the dataset
        for var_name in xds_renamed.data_vars:
            # Convert the data type of the variable to float32
            xds_renamed[var_name] = xds_renamed[var_name].astype('float32')
        
        #Converting x,y to float64 
        xds_renamed['latitude'] = xds_renamed['latitude'].astype('float32')
        xds_renamed['longitude'] = xds_renamed['longitude'].astype('float32')
        
        #Saving file
        # Extract the date part from the datetime object
        # Assuming image_date_str is a Pandas Timestamp object
        
        # Extract only the date portion
        date_only = date.date()
        
        # Convert the date back to string format if needed
        date_only_str = str(date_only)

        # Construct the filename
        filename = f"{station_name}_{date_only_str}_InitialCondition.nc"
        print(filename)
        # Save the xarray Dataset to NetCDF format
        xds_renamed.to_netcdf(os.path.join(output_path,filename))
        
    

Processing BE-Lon
2012-01-01 00:00:00
BE-Lon_2012-01-01_InitialCondition.nc
Processing CZ-KrP
2014-01-01 00:00:00
CZ-KrP_2014-01-01_InitialCondition.nc
Processing FI-Qvd
2018-01-01 00:00:00
FI-Qvd_2018-01-01_InitialCondition.nc
Processing FR-Aur
2012-01-01 00:00:00
FR-Aur_2012-01-01_InitialCondition.nc
Processing FR-Gri
2012-01-01 00:00:00
FR-Gri_2012-01-01_InitialCondition.nc
Processing DE-RuS
2012-01-01 00:00:00
DE-RuS_2012-01-01_InitialCondition.nc
Processing DK-Sor
2012-01-01 00:00:00
DK-Sor_2012-01-01_InitialCondition.nc
Processing FR-Fon
2012-01-01 00:00:00
FR-Fon_2012-01-01_InitialCondition.nc
Processing FR-Hes
2014-01-01 00:00:00
FR-Hes_2014-01-01_InitialCondition.nc
Processing DE-Hai
2012-01-01 00:00:00
DE-Hai_2012-01-01_InitialCondition.nc
Processing FI-Let
2012-01-01 00:00:00
FI-Let_2012-01-01_InitialCondition.nc
Processing FI-Var
2016-01-01 00:00:00
FI-Var_2016-01-01_InitialCondition.nc
Processing FR-Bil
2014-01-01 00:00:00
FR-Bil_2014-01-01_InitialCondition.nc
Processing D

In [9]:
xds

<xarray.Dataset>
Dimensions:                        ()
Coordinates:
    latitude                       float64 37.91
    longitude                      float64 -3.228
    time                           datetime64[ns] 2014-01-01
Data variables:
    skin_temperature               float64 277.3
    soil_temperature_level_1       float64 277.6
    soil_temperature_level_2       float64 278.2
    soil_temperature_level_3       float64 281.5
    soil_temperature_level_4       float64 287.6
    volumetric_soil_water_layer_1  float64 0.3457
    volumetric_soil_water_layer_2  float64 0.3497
    volumetric_soil_water_layer_3  float64 0.2422
    volumetric_soil_water_layer_4  float64 0.3187

In [22]:
station_names = []
for index, row in era5land.iterrows():
    # Convert the row to a dictionary
    row_dict = row.to_dict()

    station_names.append(index)

Inside each folder I want to create "config_file.txt" file, which is in following fomrat:
WorkDir=/home/khanalp/STEMMUSSCOPE/STEMMUS_SCOPE/example/ (Here I want to change to /home/khanalp/STEMMUSSCOPE/STEMMUS_SCOPE/each station name folder)
SoilPropertyPath=/home/khanalp/STEMMUSSCOPE/STEMMUS_SCOPE/input/SoilProperty/ # This exists same 
ForcingPath=/home/khanalp/task1/data/ICOS/Input_data/ # This exists same. 
Location=BE-Lon # This comes from corresponding station name. 
directional=/home/khanalp/STEMMUSSCOPE/STEMMUS_SCOPE/input/directional/ #This is same. 
fluspect_parameters=/home/khanalp/STEMMUSSCOPE/STEMMUS_SCOPE/input/fluspect_parameters/ #this is same.
leafangles=/home/khanalp/STEMMUSSCOPE/STEMMUS_SCOPE/input/leafangles/ #this is same. 
radiationdata=/home/khanalp/STEMMUSSCOPE/STEMMUS_SCOPE/input/radiationdata/ #this is same.
soil_spectrum=/home/khanalp/STEMMUSSCOPE/STEMMUS_SCOPE/input/soil_spectrum/ #this is same. 
input_data=/home/khanalp/STEMMUSSCOPE/STEMMUS_SCOPE/input/input_data.xlsx #this is same. 
InitialConditionPath=/home/khanalp/task1/data/ICOS/Initial_condition/ #this is same. 
StartTime=2012-01-01T00:00 # I want this to read from the xdataset of each input file with station name located in folder /home/khanalp/task1/data/ICOS/Input_data/ . 
EndTime=2020-12-31T23:30 # also same for this.
InputPath=
OutputPath=

In [ ]:
import os
import pandas as pd

In [ ]:
# Assuming input_data.xlsx contains start and end times for each station
input_data_path = "/home/khanalp/task1/data/ICOS/Input_data/input_data.xlsx"

# Reading the input data file
input_data = pd.read_excel(input_data_path, index_col=0)

# Assuming config file template
config_template = f"""\
WorkDir=/home/khanalp/STEMMUSSCOPE/STEMMUS_SCOPE/ICOS_sites/{station_name}/
SoilPropertyPath=/home/khanalp/STEMMUSSCOPE/STEMMUS_SCOPE/input/SoilProperty/
ForcingPath=/home/khanalp/task1/data/ICOS/Input_data/
Location={station_name}
directional=/home/khanalp/STEMMUSSCOPE/STEMMUS_SCOPE/input/directional/
fluspect_parameters=/home/khanalp/STEMMUSSCOPE/STEMMUS_SCOPE/input/fluspect_parameters/
leafangles=/home/khanalp/STEMMUSSCOPE/STEMMUS_SCOPE/input/leafangles/
radiationdata=/home/khanalp/STEMMUSSCOPE/STEMMUS_SCOPE/input/radiationdata/
soil_spectrum=/home/khanalp/STEMMUSSCOPE/STEMMUS_SCOPE/input/soil_spectrum/
input_data=/home/khanalp/STEMMUSSCOPE/STEMMUS_SCOPE/input/input_data.xlsx
InitialConditionPath=/home/khanalp/task1/data/ICOS/Initial_condition/
StartTime={input_data.loc[station_name, 'Start_Time']}
EndTime={input_data.loc[station_name, 'End_Time']}
InputPath=
OutputPath=
"""

# Change directory to the desired location
os.chdir("~/STEMMUSSCOPE/STEMMUS_SCOPE/ICOS_sites")

# Create config file for each station
for station_name in station_names:
    station_dir = f"./{station_name}"
    # Create station folder if it doesn't exist
    if not os.path.exists(station_dir):
        os.makedirs(station_dir)

    # Write config file
    with open(os.path.join(station_dir, "config_file.txt"), "w") as file:
        file.write(config_template)


In [69]:
zbcd = xr.open_dataset("/home/khanalp/task1/data/ICOS/Input_data/BE-Lon_2012-2020_FLUXNET2015_Met.nc")